In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('course_lead_scoring.csv')

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [5]:
numeric = ['number_of_courses_viewed','annual_income', 'interaction_count', 'lead_score']
categoric = ['lead_source', 'industry', 'employment_status', 'location']

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
df.loc[:, categoric] = df[categoric].fillna('NA')
df.loc[:, numeric] = df[numeric].fillna(0.0)

In [8]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [9]:
df.industry.mode()

0    retail
Name: industry, dtype: object

**Seeing corr metrics**

In [10]:
df[numeric].corrwith(df.converted)

number_of_courses_viewed    0.435914
annual_income               0.053131
interaction_count           0.374573
lead_score                  0.193673
dtype: float64

**Splitting the data**

In [11]:
X = df.drop(columns=['converted'])
y = df.converted.values

In [12]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.6, random_state=42, shuffle=True)

In [13]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=0.5, random_state=42, shuffle=True)

**Checking Mutual Information**

In [14]:
def mi_scores(series):
    return mutual_info_score(series, df.converted)

In [15]:
mi = df[categoric].apply(mi_scores)
mi.sort_values(ascending=False)

lead_source          0.026574
employment_status    0.011070
industry             0.007267
location             0.001427
dtype: float64

**Apply One Hot Encoding**

In [16]:
features = categoric + numeric

In [17]:
train_dicts = X_train[features].to_dict(orient='records')

In [18]:
dv = DictVectorizer(sparse=False)

In [19]:
X_train = dv.fit_transform(train_dicts)

In [20]:
X_train.shape

(877, 31)

In [21]:
val_dicts = X_val[features].to_dict(orient='records')

In [22]:
X_val = dv.transform(val_dicts)

In [23]:
X_val.shape

(292, 31)

**Train the Model**

In [24]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [25]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [27]:
y_pred = model.predict(X_val)
y_pred

array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [28]:
(y_val == y_pred).mean()

np.float64(0.7431506849315068)

**Get the feature importance**

In [42]:
coef = model.coef_[0]

In [45]:
feature_importance = pd.DataFrame({
    'feature': dv.get_feature_names_out(),
    'coefficient': coef,
})

In [47]:
feature_importance.sort_values(by='coefficient')

,feature,coefficient
5,employment_status=unemployed,-0.106174
19,lead_source=paid_ads,-0.095865
8,industry=finance,-0.024975
9,industry=healthcare,-0.021530
28,location=north_america,-0.021052
6,industry=NA,-0.021008
29,location=south_america,-0.018004
12,industry=retail,-0.016084
21,lead_source=social_media,-0.015059
17,lead_source=events,-0.014460


**Train the model again**

In [52]:
C_val = [0.01, 0.1, 1, 10, 100]
res = []

In [55]:
for C in C_val:
    model = LogisticRegression(
        solver='liblinear', 
        C=C, 
        max_iter=1000, 
        random_state=42
    )

    # Train the model using all features in X_train
    model.fit(X_train, y_train)

    # Predict class labels on the validation set
    y_pred = model.predict(X_val)

    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
    
    # Round the accuracy to 3 decimal digits
    #rounded_accuracy = round(accuracy, 3)

    # Store the results
    res.append({
        'C': C,
        'Validation Accuracy': rounded_accuracy
    })
    
    print(f"C={C:<6} -> Accuracy: {rounded_accuracy}")

C=0.01   -> Accuracy: 0.743
C=0.1    -> Accuracy: 0.743
C=1      -> Accuracy: 0.743
C=10     -> Accuracy: 0.743
C=100    -> Accuracy: 0.743
